In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext rpy2.ipython

In [5]:
import numpy as np
import pandas as pd

from kaggle.house_prices.utils import Plot

In [6]:
from rpy2 import robjects
from rpy2.robjects.lib import ggplot2

robjects.pandas2ri.activate()

In [16]:
from kaggle.house_prices import helpers
from kaggle.house_prices import outliers

combined_dataset = helpers.load_data()
training_dataset = combined_dataset.query("dataSource == 'train'")

In [22]:
#check some basic invariants

combined_strange_cases = outliers.get_strange_cases(combined_dataset)
combined_outliers_removed = outliers.remove_outliers(combined_dataset)

training_strange_cases = outliers.get_strange_cases(training_dataset)
training_outliers_removed = outliers.remove_outliers(training_dataset)

assert len(combined_strange_cases) + len(combined_outliers_removed) == len(combined_dataset)
assert len(training_strange_cases) + len(training_outliers_removed) == len(training_dataset)
pd.testing.assert_frame_equal(combined_strange_cases, training_strange_cases)

In [28]:
(
    training_dataset
    [['Neighborhood', 'SalePrice']]
).head()

x = (
    training_dataset
    .groupby('Neighborhood')
    ['SalePrice']
    .apply(lambda x: np.median(x))
)

x

,Neighborhood,SalePrice
0,CollgCr,208500.0
1,Veenker,181500.0
2,CollgCr,223500.0
3,Crawfor,140000.0
4,NoRidge,250000.0


Neighborhood
Blmngtn    191000.0
Blueste    137500.0
BrDale     106000.0
BrkSide    124300.0
ClearCr    200250.0
CollgCr    197200.0
Crawfor    200624.0
Edwards    121750.0
Gilbert    181000.0
IDOTRR     103000.0
MeadowV     88000.0
Mitchel    153500.0
NAmes      140000.0
NPkVill    146000.0
NWAmes     182900.0
NoRidge    301500.0
NridgHt    315000.0
OldTown    119000.0
SWISU      139500.0
Sawyer     135000.0
SawyerW    179900.0
Somerst    225500.0
StoneBr    278000.0
Timber     228475.0
Veenker    218000.0
Name: SalePrice, dtype: float64

In [ ]:
#
# SPDANM - deviation of price from neighborhood's median price
# price_per_sq - price per living area square feet
#

def calc_quality(df):
    q = df['OverallQual']
    if q in (1, 2, 3):
        return 'Bad'
    if q in (4, 5, 6):
        return 'Fair'
    if q in (7, 8):
        return 'Good'
    return 'Excellent'

def calc_SPDANM(df):
    

training_dataset = (
    training_dataset
    .assign(
        quality=calc_quality,
        price_per_sq=lambda df: df['SalePrice'] / df['GrLivArea'],
        SPDANM=lambda df: (
            df
            .groupby('Neighborhood')
            ['SalePrice']
            .apply(lambda x: np.median(x))
        )
    )
    .groupby('Neighborhood', as_index = False)
    .assign(
    )
    mutate(
        quality = case_when(
            OverallQual %in% c(1, 2, 3) ~ 'Bad',
            OverallQual %in% c(4, 5, 6) ~ 'Fair',
            OverallQual %in% c(7, 8)    ~ 'Good',
            TRUE                        ~ 'Excellent'
        ),
        price_per_sq = SalePrice / GrLivArea
    ) %>%
    group_by(Neighborhood) %>%
    mutate(
        SPDANM = SalePrice - median(SalePrice)
    )
) 
    

strange_cases <- house_prices$outliers$get_strange_cases(training_dataset)
EdwardsExcellent <- training_dataset %>% filter(Neighborhood == 'Edwards', quality == 'Excellent')